In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
import math

In [ ]:
feature_imp_ohe = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/rf_importances.csv')
feature_imp_label = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/rf_importances_label.csv')

train_ohe = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/train_tree.csv')
train_ohe = train_ohe[~train_ohe["Approved_Flag"].isna()]
train_label = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/train_label.csv')
train_label = train_label[~train_label["Approved_Flag"].isna()]
train_lr = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/train_lr.csv')
train_lr = train_lr[~train_lr["Approved_Flag"].isna()]

test_ohe = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/test_tree.csv')
test_label = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/test_label.csv')
test_ohe = test_ohe[~test_ohe["Approved_Flag"].isna()]
test_label = test_label[~test_label["Approved_Flag"].isna()]
test_lr = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/test_lr.csv')
test_lr = test_lr[~test_lr["Approved_Flag"].isna()]

test_set_index = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/test_set_index.csv')
test_set_index = test_set_index["index"].to_list()

test_ohe_100 = test_ohe.loc[test_set_index]
test_label_100 = test_label.loc[test_set_index]
test_lr_100 = test_lr.loc[test_set_index]

In [ ]:
def evaluate_models_with_tuning(
    train_ohe,
    train_label,
    train_lr,
    test_ohe_100,
    test_label_100,
    test_lr_100,
    sizes,
    random_state=42,
    hyperparameter_tuning=False
):
    results = []

    # Define hyperparameter grids
    param_grids = {
        "RandomForest": {
            "n_estimators": [10, 50, 100, 200],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
        },
        "DecisionTree": {
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
        },
        "XGBoost": {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 5, 10],
            "learning_rate": [0.01, 0.1, 0.2],
            "subsample": [0.6, 0.8, 1.0],
        },
    }

    # Iterate over training sizes
    for size in sizes:
        # Sampling training data
        stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=len(train_ohe) - size, random_state=random_state)
        train_idx, _ = next(stratified_split.split(train_ohe, train_ohe["Approved_Flag"]))

        sampled_train_ohe = train_ohe.iloc[train_idx]

        stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=len(train_lr) - size, random_state=random_state)
        train_idx, _ = next(stratified_split.split(train_lr, train_lr["Approved_Flag"]))
        sampled_train_lr = train_lr.iloc[train_idx]
        # sampled_indices_ohe = train_ohe.sample(size).index
        # sampled_train_ohe = train_ohe.loc[sampled_indices_ohe]
        # sampled_indices_lr = train_lr.sample(size).index
        # sampled_train_lr = train_lr.loc[sampled_indices_lr]

        X_train_tree = sampled_train_ohe.drop(columns="Approved_Flag")
        y_train_tree = sampled_train_ohe["Approved_Flag"]
        X_train_lr = sampled_train_lr.drop(columns="Approved_Flag")
        y_train_lr = sampled_train_lr["Approved_Flag"]

        X_test_tree = test_ohe_100.drop(columns="Approved_Flag")
        y_test_tree = test_label_100["Approved_Flag"]
        X_test_lr = test_lr_100.drop(columns="Approved_Flag")
        y_test_lr = test_lr_100["Approved_Flag"]

        # Models
        models_tree = {
            "RandomForest": RandomForestClassifier(),
            "DecisionTree": DecisionTreeClassifier(max_depth=int(math.log2(len(train_idx)))),
            "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
        }
        model_lr = LogisticRegression(max_iter=1000)

        # Train and evaluate tree-based models
        for model_name, model in models_tree.items():
            if hyperparameter_tuning:
                # Perform RandomizedSearchCV
                search = RandomizedSearchCV(
                    estimator=model,
                    param_distributions=param_grids[model_name],
                    n_iter=10,
                    scoring="roc_auc",
                    cv=3,
                    random_state=42,
                    n_jobs=-1
                )
                search.fit(X_train_tree, y_train_tree)
                model = search.best_estimator_
            else:
                model.fit(X_train_tree, y_train_tree)

            preds = model.predict(X_test_tree)
            if hasattr(model, "predict_proba"):
                probs = model.predict_proba(X_test_tree)
                if probs.shape[1] > 1:
                    probs = probs[:, 1]  # Take probabilities of the positive class
                else:
                    probs = probs[:, 0]  # Single column output
            else:
                # Use predictions as probabilities for models without `predict_proba`
                probs = preds
            print(model_name)
            print("preds")
            print(preds)
            print("probs")
            print(probs)
            results.append({
                "Model": model_name,
                "Train Size": size,
                "F1": f1_score(y_test_tree, preds),
                "ROCAUC": roc_auc_score(y_test_tree, probs),
                "PRAUC": average_precision_score(y_test_tree, probs)
            })

        # Train and evaluate logistic regression
        model_lr.fit(X_train_lr, y_train_lr)
        preds = model_lr.predict(X_test_lr)
        probs = model_lr.predict_proba(X_test_lr)[:, 1]  # Logistic regression always outputs two classes
        print("LR")
        print("preds")
        print(preds)
        print("probs")
        print(probs)
        results.append({
            "Model": "LogisticRegression",
            "Train Size": size,
            "F1": f1_score(y_test_lr, preds),
            "ROCAUC": roc_auc_score(y_test_lr, probs),
            "PRAUC": average_precision_score(y_test_lr, probs)
        })

    return pd.DataFrame(results)


In [ ]:
# Initialize an empty list to store results
all_results = []
train_sizes = [128]
# Define number of repetitions
n_repeats = 10

# Run evaluation 10 times for both without and with hyperparameter tuning
for i in range(n_repeats):
    # Evaluate without hyperparameter tuning
    results_no_tuning = evaluate_models_with_tuning(
        train_ohe,
        train_label,
        train_lr,
        test_ohe_100,
        test_label_100,
        test_lr_100,
        train_sizes,
        random_state=i,
        hyperparameter_tuning=False
    )
    # Add iteration number and tuning type
    results_no_tuning["Iteration"] = i + 1
    results_no_tuning["Tuning"] = "No"

    # Evaluate with hyperparameter tuning
    results_with_tuning = evaluate_models_with_tuning(
        train_ohe,
        train_label,
        train_lr,
        test_ohe_100,
        test_label_100,
        test_lr_100,
        train_sizes,
        random_state=i,
        hyperparameter_tuning=True
    )
    # Add iteration number and tuning type
    results_with_tuning["Iteration"] = i + 1
    results_with_tuning["Tuning"] = "Yes"

    # Append both results to the list
    all_results.append(results_no_tuning)
    all_results.append(results_with_tuning)

# Combine all results into a single DataFrame
final_results = pd.concat(all_results, ignore_index=True)

RandomForest
preds
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
probs
[0.07 0.15 0.07 0.21 0.09 0.14 0.18 0.1  0.15 0.07 0.12 0.09 0.07 0.07
 0.17 0.13 0.1  0.15 0.21 0.21 0.1  0.17 0.12 0.09 0.09 0.12 0.1  0.14
 0.12 0.04 0.14 0.18 0.13 0.25 0.1  0.12 0.25 0.2  0.1  0.06 0.15 0.18
 0.08 0.07 0.14 0.11 0.14 0.12 0.08 0.15 0.15 0.16 0.18 0.1  0.18 0.1
 0.16 0.1  0.13 0.15 0.24 0.16 0.06 0.07 0.06 0.33 0.12 0.09 0.09 0.14
 0.2  0.16 0.29 0.11 0.11 0.04 0.14 0.09 0.2  0.14 0.07 0.12 0.12 0.11
 0.13 0.13 0.15 0.08 0.07 0.17 0.1  0.12 0.11 0.19 0.08 0.12 0.09 0.22
 0.22 0.13]
DecisionTree
preds
[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:49:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost
preds
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
probs
[0.06078685 0.02487194 0.13593777 0.07653569 0.00520674 0.04359728
 0.16077524 0.20728843 0.00683431 0.00146924 0.05650834 0.0067746
 0.01422152 0.00327899 0.01454899 0.13898137 0.2553521  0.05789257
 0.07634473 0.02634259 0.05429378 0.03148732 0.03050511 0.03584094
 0.00493483 0.74746805 0.02799622 0.01468639 0.00863943 0.00148907
 0.17073953 0.03251158 0.00514818 0.11814272 0.01085432 0.01360912
 0.7670711  0.03388792 0.0307399  0.03005156 0.03509373 0.01869144
 0.00881608 0.01572339 0.0164246  0.00238059 0.04603942 0.02500979
 0.01662603 0.00566351 0.05272297 0.14639817 0.04646521 0.00455176
 0.00238495 0.007387   0.14076935 0.00269493 0.4727828  0.01304678
 0.01188248 0.05392817 0.08409036 0.00382513 0.00991333 0.18322629
 0.00600702 0.0075989  0.0025959  0.0112

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR
preds
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0.]
probs
[5.54189778e-02 5.12019708e-02 6.38822718e-02 8.00336922e-01
 3.57801449e-02 2.10360302e-02 5.25566775e-02 2.82547956e-03
 4.95529315e-02 1.11055759e-01 1.00476376e-03 3.54933971e-02
 7.36014994e-03 3.82496895e-02 1.70084201e-03 1.03545331e-01
 7.03048374e-02 1.62260189e-01 1.04347337e-01 2.02043112e-02
 6.37552058e-02 0.00000000e+00 1.40075209e-01 3.60770195e-02
 7.61116161e-02 1.08686054e-01 1.83270828e-01 5.51258087e-01
 0.00000000e+00 7.31139715e-03 1.16455734e-01 3.42654316e-03
 1.38726600e-02 3.00570564e-03 1.95143868e-01 2.35447406e-03
 0.00000000e+00 9.82304243e-01 7.42792973e-02 3.47472863e-02
 1.68054651e-01 2.07319441e-01 5.27087984e-02 5.14701147e-02
 1.05356

...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR
preds
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
probs
[9.41969609e-03 1.86080058e-02 8.88324079e-01 4.00331405e-02
 3.48127543e-01 4.50573283e-02 6.97827149e-03 1.32292054e-01
 0.00000000e+00 9.36153153e-04 4.41827377e-02 2.69905733e-01
 0.00000000e+00 6.06310581e-02 2.27602573e-02 1.05871815e-01
 2.56172968e-04 1.51734242e-01 8.59989082e-02 1.30402004e-03
 2.67127370e-01 1.22077261e-03 5.04539808e-01 0.00000000e+00
 7.74875056e-02 1.03670804e-01 2.18853041e-01 0.00000000e+00
 7.36501915e-01 1.92230868e-02 3.18329815e-01 0.00000000e+00
 4.28200057e-02 0.00000000e+00 1.86308925e-01 0.00000000e+00
 8.12041779e-01 0.00000000e+00 2.76518396e-01 5.82696730e-03
 0.00000000e+00 6.46659922e-02 1.72447101e-01 5.57245000e-03
 8.46280

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:50:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost
preds
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
probs
[9.08774149e-04 6.09151379e-04 5.85050147e-04 4.05345894e-02
 2.84819189e-03 4.11677659e-01 2.11542146e-03 4.68810834e-03
 5.66503638e-03 2.11379789e-02 1.22092582e-01 3.25970054e-02
 2.60881096e-01 7.36494362e-03 5.84841613e-03 4.44931909e-02
 8.00116360e-02 5.69426501e-03 1.15674675e-01 2.02112764e-01
 1.70254742e-03 1.18285164e-01 1.48029570e-02 4.63420554e-04
 4.84406948e-03 2.68807113e-02 9.76354539e-01 3.05227578e-01
 6.34385794e-02 4.39130468e-03 1.91079881e-02 3.71708423e-02
 6.70230165e-02 5.49137825e-04 5.57832466e-03 1.18464103e-03
 1.11787794e-02 1.42835465e-03 2.80659436e-03 3.02615133e-03
 3.65757104e-03 3.69124822e-02 4.76393662e-02 1.17217412e-03
 4.10617003e-03 1.71993382e-03 8.68153498e-02 1.08019579e-02
 7.50693376e-04 1.24098165e-02 1.36763960e-0

KeyboardInterrupt: 

In [ ]:
final_results.to_csv('/content/drive/MyDrive/Indian_bank_data/traiditional_ML_results.csv', index=False)

In [ ]:
final_results

,Model,Train Size,F1,ROCAUC,PRAUC,Iteration,Tuning
0,RandomForest,8,0.000000,0.316534,0.102627,1,No
1,DecisionTree,8,0.000000,0.500000,0.130000,1,No
2,XGBoost,8,0.000000,0.500000,0.130000,1,No
3,LogisticRegression,8,0.200000,0.607869,0.189695,1,No
4,RandomForest,16,0.000000,0.359859,0.110378,1,No
...,...,...,...,...,...,...,...
395,LogisticRegression,64,0.230769,0.558355,0.254256,10,Yes
396,RandomForest,128,0.000000,0.641026,0.181287,10,Yes
397,DecisionTree,128,0.263158,0.588417,0.176154,10,Yes
398,XGBoost,128,0.111111,0.497790,0.190122,10,Yes


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Run

In [ ]:
final_results.groupby(["Model", "Tuning", "Train Size"]).mean()

F1    ROCAUC     PRAUC  Iteration
Model              Tuning Train Size                                         
DecisionTree       No     8           0.084405  0.505747  0.137094        5.5
                          16          0.054040  0.480592  0.136000        5.5
                          32          0.093360  0.497701  0.133870        5.5
                          64          0.073252  0.485455  0.134953        5.5
                          128         0.132694  0.499602  0.136777        5.5
                   Yes    8           0.000000  0.500000  0.130000        5.5
                          16          0.000000  0.520336  0.147548        5.5
                          32          0.045652  0.530592  0.142386        5.5
                          64          0.081705  0.497657  0.141580        5.5
                          128         0.094865  0.497038  0.142852        5.5
LogisticRegression No     8           0.253362  0.597215  0.200022        5.5
                          16          0.201326  0.529310  0.211544        5.5
                          32          0.113592  0.376569  0.177702        5.5
                          64          0.115729  0.450575  0.190337        5.5
                          128         0.092260  0.462025  0.168424        5.5
                   Yes    8           0.253362  0.597215  0.200022        5.5
                          16          0.201326  0.529310  0.211544        5.5
                          32          0.113592  0.376569  0.177702        5.5
                          64          0.115729  0.450575  0.190337        5.5
                          128         0.092260  0.462025  0.168424        5.5
RandomForest       No     8           0.000000  0.531123  0.161334        5.5
                          16          0.000000  0.542352  0.156961        5.5
                          32          0.000000  0.528205  0.174340        5.5
                          64          0.014286  0.529089  0.178624        5.5
                          128         0.000000  0.550000  0.195730        5.5
                   Yes    8           0.000000  0.500000  0.130000        5.5
                          16          0.000000  0.471397  0.151625        5.5
                          32          0.000000  0.524226  0.197860        5.5
                          64          0.000000  0.485102  0.157079        5.5
                          128         0.000000  0.523077  0.169660        5.5
XGBoost            No     8           0.000000  0.500000  0.130000        5.5
                          16          0.000000  0.464545  0.136762        5.5
                          32          0.041863  0.442971  0.149276        5.5
                          64          0.047235  0.490407  0.146227        5.5
                          128         0.058952  0.500531  0.157032        5.5
                   Yes    8           0.000000  0.500000  0.130000        5.5
                          16          0.000000  0.493457  0.129427        5.5
                          32          0.023810  0.465606  0.151868        5.5
                          64          0.019048  0.468612  0.148889        5.5
                          128         0.052822  0.504819  0.172024        5.5

In [ ]:
def evaluate_models_with_tuning_adjusted_proportion(
    train_ohe,
    train_label,
    train_lr,
    test_ohe_100,
    test_label_100,
    test_lr_100,
    sizes,
    minority_proportions,
    random_state=42,
    hyperparameter_tuning=False
):
    from sklearn.utils import resample

    results = []

    # Define hyperparameter grids
    param_grids = {
        "RandomForest": {
            "n_estimators": [10, 50, 100, 200],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
        },
        "DecisionTree": {
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
        },
        "XGBoost": {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 5, 10],
            "learning_rate": [0.01, 0.1, 0.2],
            "subsample": [0.6, 0.8, 1.0],
        },
    }

    # Iterate over minority proportions
    for minority_proportion in minority_proportions:
        # Iterate over training sizes
        for size in sizes:
            # Separate majority and minority classes
            majority = train_ohe[train_ohe["Approved_Flag"] == 0]
            minority = train_ohe[train_ohe["Approved_Flag"] == 1]

            # Calculate the number of samples needed for each class
            n_minority = int(size * minority_proportion)
            n_majority = size - n_minority

            # Sample data
            sampled_minority = resample(minority, replace=True, n_samples=n_minority, random_state=random_state)
            sampled_majority = resample(majority, replace=True, n_samples=n_majority, random_state=random_state)
            sampled_train_ohe = pd.concat([sampled_minority, sampled_majority])

            majority = train_lr[train_lr["Approved_Flag"] == 0]
            minority = train_lr[train_lr["Approved_Flag"] == 1]

            # Calculate the number of samples needed for each class
            n_minority = int(size * minority_proportion)
            n_majority = size - n_minority

            # Sample data
            sampled_minority = resample(minority, replace=True, n_samples=n_minority, random_state=random_state)
            sampled_majority = resample(majority, replace=True, n_samples=n_majority, random_state=random_state)
            sampled_train_lr = pd.concat([sampled_minority, sampled_majority])

            # Split features and labels
            X_train_tree = sampled_train_ohe.drop(columns="Approved_Flag")
            y_train_tree = sampled_train_ohe["Approved_Flag"]
            X_train_lr = sampled_train_lr.drop(columns="Approved_Flag")
            y_train_lr = sampled_train_lr["Approved_Flag"]

            X_test_tree = test_ohe_100.drop(columns="Approved_Flag")
            y_test_tree = test_ohe_100["Approved_Flag"]
            X_test_lr = test_lr_100.drop(columns="Approved_Flag")
            y_test_lr = test_lr_100["Approved_Flag"]

            # Models
            models_tree = {
                "RandomForest": RandomForestClassifier(),
                "DecisionTree": DecisionTreeClassifier(max_depth = int(math.log2(len(X_train_tree)))),
                "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
            }
            model_lr = LogisticRegression(max_iter=1000)

            # Train and evaluate tree-based models
            for model_name, model in models_tree.items():
                if hyperparameter_tuning:
                    # Perform RandomizedSearchCV
                    search = RandomizedSearchCV(
                        estimator=model,
                        param_distributions=param_grids[model_name],
                        n_iter=10,
                        scoring="roc_auc",
                        cv=3,
                        random_state=42,
                        n_jobs=-1
                    )
                    search.fit(X_train_tree, y_train_tree)
                    model = search.best_estimator_
                else:
                    model.fit(X_train_tree, y_train_tree)

                preds = model.predict(X_test_tree)
                if hasattr(model, "predict_proba"):
                    probs = model.predict_proba(X_test_tree)
                    if probs.shape[1] > 1:
                        probs = probs[:, 1]  # Take probabilities of the positive class
                    else:
                        probs = probs[:, 0]  # Single column output
                else:
                    # Use predictions as probabilities for models without `predict_proba`
                    probs = preds
                print(model_name)
                print("preds")
                print(preds)
                print("probs")
                print(probs)
                results.append({
                    "Model": model_name,
                    "Train Size": size,
                    "Class 1 Proportion": minority_proportion,
                    "F1_Score": f1_score(y_test_lr, preds),
                    "ROC_AUC": roc_auc_score(y_test_lr, probs),
                    "PR_AUC": average_precision_score(y_test_lr, probs)
                })

            # Train and evaluate logistic regression
            model_lr.fit(X_train_lr, y_train_lr)
            preds = model_lr.predict(X_test_lr)
            probs = model_lr.predict_proba(X_test_lr)[:, 1]  # Logistic regression always outputs two classes
            results.append({
                "Model": "LogisticRegression",
                "Train Size": size,
                "Class 1 Proportion": minority_proportion,
                "F1_Score": f1_score(y_test_lr, preds),
                "ROC_AUC": roc_auc_score(y_test_lr, probs),
                "PR_AUC": average_precision_score(y_test_lr, probs)
            })

    return pd.DataFrame(results)


In [ ]:
# Initialize an empty list to store results
all_results = []
train_sizes = [32, 64, 128]
# Define number of repetitions
n_repeats = 10

# Run evaluation 10 times for both without and with hyperparameter tuning
for i in range(n_repeats):
    # Evaluate without hyperparameter tuning
    results_no_tuning = evaluate_models_with_tuning_adjusted_proportion(
        train_ohe,
        train_label,
        train_lr,
        test_ohe_100,
        test_label_100,
        test_lr_100,
        train_sizes,
        [0.3, 0.5],
        random_state=i,
        hyperparameter_tuning=False
    )
    # Add iteration number and tuning type
    results_no_tuning["Iteration"] = i + 1
    results_no_tuning["Tuning"] = "No"

    # Evaluate with hyperparameter tuning
    results_with_tuning = evaluate_models_with_tuning_adjusted_proportion(
        train_ohe,
        train_label,
        train_lr,
        test_ohe_100,
        test_label_100,
        test_lr_100,
        train_sizes,
        [0.3, 0.5],
        random_state=i,
        hyperparameter_tuning=True
    )
    # Add iteration number and tuning type
    results_with_tuning["Iteration"] = i + 1
    results_with_tuning["Tuning"] = "Yes"

    # Append both results to the list
    all_results.append(results_no_tuning)
    all_results.append(results_with_tuning)

# Combine all results into a single DataFrame
final_results = pd.concat(all_results, ignore_index=True)

RandomForest
preds
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
probs
[0.31 0.2  0.35 0.3  0.28 0.28 0.24 0.23 0.34 0.35 0.31 0.43 0.25 0.25
 0.27 0.18 0.26 0.32 0.15 0.2  0.33 0.48 0.26 0.51 0.31 0.27 0.24 0.4
 0.4  0.26 0.19 0.44 0.18 0.44 0.25 0.29 0.42 0.39 0.5  0.38 0.41 0.21
 0.16 0.33 0.41 0.4  0.4  0.47 0.47 0.23 0.24 0.18 0.32 0.25 0.27 0.33
 0.2  0.32 0.24 0.19 0.43 0.15 0.41 0.2  0.44 0.39 0.21 0.25 0.28 0.18
 0.25 0.2  0.35 0.35 0.29 0.25 0.4  0.25 0.43 0.18 0.29 0.43 0.34 0.23
 0.3  0.31 0.2  0.35 0.21 0.23 0.32 0.24 0.29 0.18 0.3  0.47 0.19 0.3
 0.39 0.42]
DecisionTree
preds
[1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:55:10] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost
preds
[0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0
 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1
 1 0 0 0 1 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1]
probs
[0.01985428 0.2631648  0.88553274 0.09814141 0.0868675  0.03553579
 0.45171055 0.34271798 0.97550356 0.5408521  0.05412148 0.25580734
 0.01761309 0.17904133 0.00511659 0.46697593 0.5595571  0.26518604
 0.00705142 0.02566805 0.40820596 0.86894166 0.03063422 0.7174472
 0.01911821 0.05711358 0.02563405 0.9779154  0.64799273 0.00449616
 0.00680419 0.37040442 0.01491363 0.7608486  0.02188049 0.12889086
 0.19968511 0.50015074 0.28922176 0.61531365 0.7365059  0.41347548
 0.0079863  0.01970714 0.56972164 0.8727063  0.35185346 0.33678094
 0.4550718  0.18147837 0.04938785 0.00405913 0.2304483  0.05074798
 0.02913592 0.70297027 0.01687947 0.0161711  0.08956963 0.04140698
 0.5408521  0.13923284 0.54468566 0.00650204 0.8789666  0.558757
 0.05619578 0.01430679 0.46065181 0.039195

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR
preds
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 1. 0.]
probs
[3.68444882e-02 1.57299486e-02 3.18069547e-04 1.66748831e-04
 7.68990420e-03 5.97888968e-06 1.18136899e-01 1.43893035e-03
 9.97503044e-01 5.31899930e-02 4.20823121e-04 5.15518166e-05
 1.21505414e-02 8.53337997e-03 2.68872037e-02 9.65779139e-04
 9.19525625e-01 2.11189946e-01 2.02298200e-01 2.84550987e-10
 3.11900283e-03 0.00000000e+00 1.44599538e-01 1.18944721e-01
 5.93489679e-02 3.48522671e-01 1.86734680e-01 3.26247294e-01
 0.00000000e+00 1.12286498e-03 2.05512072e-01 9.97022882e-01
 4.96674543e-04 9.99976744e-01 5.86331036e-01 9.99548684e-01
 0.00000000e+00 5.33811947e-05 4.95660777e-02 3.05236160e-03
 1.27652546e-01 1.84611131e-01 1.47080431e-04 1.20464834e-02
 9.97875

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:55:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost
preds
[0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0
 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
probs
[0.03091012 0.0057664  0.74870706 0.05459811 0.28224462 0.30766013
 0.02194947 0.02291574 0.08657011 0.52875453 0.4060276  0.11568608
 0.08640945 0.41608715 0.18238746 0.49995202 0.01315264 0.3706305
 0.02048416 0.21340348 0.09837752 0.07388219 0.03024046 0.52964973
 0.25109965 0.02415079 0.05644441 0.22748156 0.37234858 0.20369962
 0.03875495 0.59989375 0.01673273 0.09925216 0.02879313 0.17897747
 0.0498778  0.8255976  0.04056026 0.64984745 0.11967008 0.44073978
 0.05544854 0.05598412 0.23748404 0.7808842  0.22024965 0.34774187
 0.22575116 0.04492358 0.01805218 0.03453641 0.63726276 0.13431679
 0.26956987 0.8482185  0.00571681 0.12468947 0.20204605 0.09082624
 0.52893126 0.10740685 0.72197545 0.08165183 0.06519756 0.6728877
 0.04203279 0.85138655 0.28046307 0.11872

KeyboardInterrupt: 

In [ ]:
final_results.to_csv('/content/drive/MyDrive/Indian_bank_data/traiditional_ML_adjusted_prop_results.csv', index=False)

In [ ]:
final_results_stratified.groupby(["Model", "Tuning", "Train Size", "Minority Proportion"]).mean()

F1    ROCAUC  \
Model              Tuning Train Size Minority Proportion                       
DecisionTree       No     8          0.1                  0.148056  0.529089   
                          16         0.1                  0.211426  0.543324   
                          32         0.1                  0.230769  0.557913   
                          64         0.1                  0.230769  0.557913   
                          128        0.1                  0.230769  0.557913   
                   Yes    8          0.1                  0.000000  0.500000   
                          16         0.1                  0.000000  0.489434   
                          32         0.1                  0.230769  0.557913   
                          64         0.1                  0.230769  0.557913   
                          128        0.1                  0.230769  0.557913   
LogisticRegression No     8          0.1                  0.253362  0.597215   
                          16         0.1                  0.201326  0.529310   
                          32         0.1                  0.113592  0.376569   
                          64         0.1                  0.115729  0.450575   
                          128        0.1                  0.092260  0.462025   
                   Yes    8          0.1                  0.253362  0.597215   
                          16         0.1                  0.201326  0.529310   
                          32         0.1                  0.113592  0.376569   
                          64         0.1                  0.115729  0.450575   
                          128        0.1                  0.092260  0.462025   
RandomForest       No     8          0.1                  0.000000  0.534350   
                          16         0.1                  0.000000  0.540009   
                          32         0.1                  0.000000  0.561848   
                          64         0.1                  0.014286  0.573254   
                          128        0.1                  0.021667  0.566180   
                   Yes    8          0.1                  0.000000  0.500000   
                          16         0.1                  0.000000  0.550884   
                          32         0.1                  0.000000  0.556233   
                          64         0.1                  0.000000  0.576879   
                          128        0.1                  0.042857  0.570203   
XGBoost            No     8          0.1                  0.000000  0.500000   
                          16         0.1                  0.000000  0.464545   
                          32         0.1                  0.041863  0.442971   
                          64         0.1                  0.230769  0.574801   
                          128        0.1                  0.230769  0.583599   
                   Yes    8          0.1                  0.000000  0.500000   
                          16         0.1                  0.000000  0.493457   
                          32         0.1                  0.023810  0.465606   
                          64         0.1                  0.226740  0.522104   
                          128        0.1                  0.230769  0.582361   

                                                             PRAUC  Iteration  
Model              Tuning Train Size Minority Proportion                       
DecisionTree       No     8          0.1                  0.144992        5.5  
                          16         0.1                  0.148450        5.5  
                          32         0.1                  0.153254        5.5  
                          64         0.1                  0.153254        5.5  
                          128        0.1                  0.153254        5.5  
                   Yes    8          0.1                  0.130000        5.5  
                          16         0.1                  0.137

In [ ]:
train_sizes = [8, 16, 32, 64, 128]

# Evaluate without hyperparameter tuning
results_no_tuning = evaluate_models_with_tuning(
    train_ohe, train_label, train_lr, test_ohe_100, test_label_100, test_lr_100, train_sizes, hyperparameter_tuning=False
)

# Evaluate with hyperparameter tuning
results_with_tuning = evaluate_models_with_tuning(
    train_ohe, train_label, train_lr, test_ohe_100, test_label_100, test_lr_100, train_sizes, hyperparameter_tuning=True
)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:48:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [18:48:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: User

In [ ]:
results_no_tuning

,Model,Train Size,F1,ROCAUC,PRAUC
0,RandomForest,8,0.000000,0.501326,0.172362
1,DecisionTree,8,0.000000,0.500000,0.130000
2,XGBoost,8,0.000000,0.500000,0.130000
3,LogisticRegression,8,0.065574,0.320955,0.094832
4,RandomForest,16,0.000000,0.541556,0.182542
5,DecisionTree,16,0.230769,0.557913,0.153254
6,XGBoost,16,0.000000,0.496021,0.157298
7,LogisticRegression,16,0.080000,0.428824,0.133497
8,RandomForest,32,0.000000,0.626437,0.226795
9,DecisionTree,32,0.230769,0.557913,0.153254


In [ ]:
results_with_tuning

,Model,Train Size,F1,ROCAUC,PRAUC
0,RandomForest,8,0.000000,0.500000,0.130000
1,DecisionTree,8,0.000000,0.500000,0.130000
2,XGBoost,8,0.000000,0.500000,0.130000
3,LogisticRegression,8,0.065574,0.320955,0.094832
4,RandomForest,16,0.000000,0.662246,0.283585
5,DecisionTree,16,0.000000,0.500000,0.130000
6,XGBoost,16,0.000000,0.500000,0.130000
7,LogisticRegression,16,0.080000,0.428824,0.133497
8,RandomForest,32,0.000000,0.549956,0.247924
9,DecisionTree,32,0.230769,0.557913,0.153254
